# Run Lineage

**E-Commerce Pipeline Lineage Analysis**


This script demonstrates how to use clgraph to analyze the SQL pipeline
in this example directory.

### Imports

In [1]:
from pathlib import Path

from clgraph import Pipeline


def load_sql_queries(sql_dir: Path) -> list[tuple[str, str]]:
    """Load all SQL files from a directory in sorted order."""
    queries = []
    for sql_file in sorted(sql_dir.glob("*.sql")):
        with open(sql_file) as f:
            sql = f.read()
        query_name = sql_file.stem
        queries.append((query_name, sql))
        print(f"  Loaded: {query_name}")
    return queries

### Code

In [2]:
print("E-Commerce Pipeline Lineage Analysis")
print()

# Load SQL files from current directory
sql_dir = Path(".")
queries = load_sql_queries(sql_dir)

print(f"\nLoaded {len(queries)} SQL files")
print()

# Build the pipeline
print("Building pipeline...")
pipeline = Pipeline(queries, dialect="duckdb")
print(f"✓ Built pipeline with {len(pipeline.table_graph.queries)} queries")
print(f"✓ Found {len(pipeline.columns)} columns")
print()

E-Commerce Pipeline Lineage Analysis

  Loaded: 01_raw_orders
  Loaded: 02_raw_customers
  Loaded: 03_raw_products
  Loaded: 04_raw_order_items
  Loaded: 05_stg_orders_enriched
  Loaded: 06_int_daily_metrics
  Loaded: 07_mart_customer_ltv
  Loaded: 08_mart_product_perf

Loaded 8 SQL files

Building pipeline...
✓ Built pipeline with 8 queries
✓ Found 326 columns



### Section

In [3]:
print("1. QUERY EXECUTION ORDER (Topologically Sorted)")
print("  Queries are sorted so dependencies come before dependents.\n")
query_order = pipeline.table_graph.topological_sort()
for i, query_id in enumerate(query_order, 1):
    query = pipeline.table_graph.queries[query_id]
    dest = query.destination_table or "(no destination)"
    print(f"  {i}. {query_id} → {dest}")
print()

1. QUERY EXECUTION ORDER (Topologically Sorted)
  Queries are sorted so dependencies come before dependents.

  1. 01_raw_orders → raw_orders
  2. 02_raw_customers → raw_customers
  3. 03_raw_products → raw_products
  4. 04_raw_order_items → raw_order_items
  5. 05_stg_orders_enriched → stg_orders_enriched
  6. 06_int_daily_metrics → int_daily_metrics
  7. 07_mart_customer_ltv → mart_customer_ltv
  8. 08_mart_product_perf → mart_product_performance



### Section

In [4]:
print("2. QUERY DEPENDENCIES (graphlib format)")
print("  Format: {query_id: {dependencies}} - used by graphlib.TopologicalSorter\n")


# Helper function to extract CTE names from SQL
def get_cte_names(sql: str) -> set:
    """Extract CTE names from a SQL query using sqlglot."""
    import sqlglot

    try:
        parsed = sqlglot.parse_one(sql, dialect="duckdb")
        cte_names = set()
        for cte in parsed.find_all(sqlglot.exp.CTE):
            if cte.alias:
                cte_names.add(cte.alias)
        return cte_names
    except Exception:
        return set()


# Build graphlib-style dependency dict
query_deps = {}
query_details = {}  # Store details for later display

for query_id in query_order:
    query = pipeline.table_graph.queries[query_id]
    cte_names = get_cte_names(query.sql)

    # Separate source tables into categories
    ctes = []
    external_sources = []
    dep_query_ids = set()

    for table_name in sorted(query.source_tables):
        if table_name in cte_names:
            ctes.append(table_name)
        else:
            table_node = pipeline.table_graph.tables.get(table_name)
            if table_node:
                if table_node.created_by and table_node.created_by != query_id:
                    dep_query_ids.add(table_node.created_by)
                elif table_node.is_source and not table_node.created_by:
                    external_sources.append(table_name)

    query_deps[query_id] = dep_query_ids
    query_details[query_id] = {
        "destination": query.destination_table,
        "ctes": ctes,
        "external_sources": external_sources,
        "dep_query_ids": dep_query_ids,
    }

# Print graphlib format
print("  dependency_graph = {")
for query_id in query_order:
    deps = query_deps[query_id]
    if deps:
        deps_str = "{" + ", ".join(f'"{d}"' for d in sorted(deps)) + "}"
    else:
        deps_str = "set()"
    print(f'      "{query_id}": {deps_str},')
print("  }")
print()

2. QUERY DEPENDENCIES (graphlib format)
  Format: {query_id: {dependencies}} - used by graphlib.TopologicalSorter

  dependency_graph = {
      "01_raw_orders": set(),
      "02_raw_customers": set(),
      "03_raw_products": set(),
      "04_raw_order_items": set(),
      "05_stg_orders_enriched": {"01_raw_orders", "02_raw_customers", "03_raw_products", "04_raw_order_items"},
      "06_int_daily_metrics": {"05_stg_orders_enriched"},
      "07_mart_customer_ltv": {"05_stg_orders_enriched"},
      "08_mart_product_perf": {"01_raw_orders", "03_raw_products", "04_raw_order_items", "05_stg_orders_enriched"},
  }



### Section

In [5]:
print("3. QUERY DETAILS")
print("  Breakdown of each query's internal structure and dependencies.\n")

for query_id in query_order:
    details = query_details[query_id]

    print(f"  {query_id}")
    print(f"    Creates table: {details['destination'] or '(none)'}")
    if details["ctes"]:
        print(f"    CTEs: {', '.join(details['ctes'])}")
    if details["external_sources"]:
        print(f"    External tables: {', '.join(details['external_sources'])}")
    if details["dep_query_ids"]:
        print(f"    Depends on queries: {', '.join(sorted(details['dep_query_ids']))}")
    if not details["ctes"] and not details["external_sources"] and not details["dep_query_ids"]:
        print("    Sources: (none)")
    print()

3. QUERY DETAILS
  Breakdown of each query's internal structure and dependencies.

  01_raw_orders
    Creates table: raw_orders
    External tables: source_orders

  02_raw_customers
    Creates table: raw_customers
    External tables: source_customers

  03_raw_products
    Creates table: raw_products
    External tables: source_products

  04_raw_order_items
    Creates table: raw_order_items
    External tables: source_order_items

  05_stg_orders_enriched
    Creates table: stg_orders_enriched
    CTEs: customer_orders_prior, order_items_agg
    Depends on queries: 01_raw_orders, 02_raw_customers, 03_raw_products, 04_raw_order_items

  06_int_daily_metrics
    Creates table: int_daily_metrics
    CTEs: daily_orders, daily_with_running
    Depends on queries: 05_stg_orders_enriched

  07_mart_customer_ltv
    Creates table: mart_customer_ltv
    CTEs: customer_metrics, customer_rfm
    Depends on queries: 05_stg_orders_enriched

  08_mart_product_perf
    Creates table: mart_produ

### Section

In [6]:
print("4. BACKWARD LINEAGE (Three levels of detail)")
print("""
graph provides three methods for backward lineage tracing:

Method                        | Returns                      | Use Case                           |
------------------------------|------------------------------|-------------------------------------|
trace_column_backward()       | Ultimate sources only        | Quick: "where does data come from?"|
get_table_lineage_path()      | Full table path (no CTEs)    | "Show complete path through tables"|
trace_column_backward_full()  | All nodes + edges (with CTEs)| Full transparency for debugging    |
""")

# Example columns to trace
examples = [
    ("mart_customer_ltv", "lifetime_revenue"),
    ("int_daily_metrics", "gross_revenue"),
    ("mart_product_performance", "total_margin"),
]

# Demonstrate all three methods for the first example
table, column = examples[0]
print(f"  Example: {table}.{column}")

# Method 1: trace_column_backward() - Ultimate sources only
print("\n  Method 1: trace_column_backward()")
print("  Returns only the ultimate source columns (leaf nodes with no upstream).\n")
try:
    sources = pipeline.trace_column_backward(table, column)
    if sources:
        print("  Code: sources = pipeline.trace_column_backward(table, column)")
        print("  Result:")
        for source in sources:
            print(f"    ← {source.table_name}.{source.column_name}")
    else:
        print("    (no sources found)")
except Exception as e:
    print(f"    Error: {e}")

# Method 2: get_table_lineage_path() - Table-level path
print("\n  Method 2: get_table_lineage_path()")
print("  Returns the complete path through real tables (skips CTEs).\n")
try:
    path = pipeline.get_table_lineage_path(table, column)
    if path:
        print("  Code: path = pipeline.get_table_lineage_path(table, column)")
        print("  Result:")
        for i, (tbl, col, query_id) in enumerate(path):
            if i == 0:
                prefix = "    "
                arrow = ""
            elif i == len(path) - 1:
                prefix = "    └── "
                arrow = "↑ "
            else:
                prefix = "    │   "
                arrow = "↑ "
            print(f"{prefix}{arrow}{tbl}.{col}")
            if i < len(path) - 1:
                print(f"    │   (from {query_id})")
    else:
        print("    (no path found)")
except Exception as e:
    print(f"    Error: {e}")

# Method 3: trace_column_backward_full() - Full transparency with CTEs
print("\n  Method 3: trace_column_backward_full()")
print("  Returns all nodes and edges, including CTEs for complete transparency.\n")
try:
    nodes, edges = pipeline.trace_column_backward_full(table, column, include_ctes=True)
    if nodes:
        print("  Code: nodes, edges = pipeline.trace_column_backward_full(table, column)")
        print("  Result (nodes):")
        for node in nodes:
            layer_tag = " [CTE]" if node.layer == "cte" else ""
            cross_query = " [cross-query]" if node.layer == "input" else ""
            print(f"    • {node.table_name}.{node.column_name}{layer_tag}{cross_query}")

        print("\n  Result (edges):")
        for edge in edges:
            edge_desc = f"({edge.edge_type})"
            print(
                f"    {edge.from_node.table_name}.{edge.from_node.column_name} "
                f"→ {edge.to_node.table_name}.{edge.to_node.column_name} {edge_desc}"
            )
    else:
        print("    (no path found)")
except Exception as e:
    print(f"    Error: {e}")

# Show additional examples with just the simple method
print("  Additional examples (using trace_column_backward for brevity):")
print()

for table, column in examples[1:]:
    print(f"  Column: {table}.{column}")
    try:
        sources = pipeline.trace_column_backward(table, column)
        if sources:
            print("  Ultimate sources:")
            for source in sources[:5]:
                print(f"    ← {source.table_name}.{source.column_name}")
            if len(sources) > 5:
                print(f"    ... and {len(sources) - 5} more")
        else:
            print("    (no sources found)")
    except Exception as e:
        print(f"    Error: {e}")
    print()

4. BACKWARD LINEAGE (Three levels of detail)

graph provides three methods for backward lineage tracing:

Method                        | Returns                      | Use Case                           |
------------------------------|------------------------------|-------------------------------------|
trace_column_backward()       | Ultimate sources only        | Quick: "where does data come from?"|
get_table_lineage_path()      | Full table path (no CTEs)    | "Show complete path through tables"|
trace_column_backward_full()  | All nodes + edges (with CTEs)| Full transparency for debugging    |

  Example: mart_customer_ltv.lifetime_revenue

  Method 1: trace_column_backward()
  Returns only the ultimate source columns (leaf nodes with no upstream).

  Code: sources = pipeline.trace_column_backward(table, column)
  Result:
    ← source_orders.total_amount

  Method 2: get_table_lineage_path()
  Returns the complete path through real tables (skips CTEs).

  Code: path = pipeline.ge

### Section

In [7]:
print("5. FORWARD LINEAGE / IMPACT ANALYSIS (Three levels of detail)")
print("""
graph provides three methods for forward lineage (impact analysis):

Method                       | Returns                      | Use Case                            |
-----------------------------|------------------------------|-------------------------------------|
trace_column_forward()       | Final impacted columns only  | Quick: "what does this affect?"     |
get_table_impact_path()      | Full table path (no CTEs)    | "Show complete impact through tables"|
trace_column_forward_full()  | All nodes + edges (with CTEs)| Full transparency for debugging     |
""")

# Example columns to trace
impact_examples = [
    ("raw_orders", "total_amount"),
    ("raw_customers", "customer_id"),
    ("raw_products", "unit_cost"),
]

# Demonstrate all three methods for the first example
table, column = impact_examples[0]
print(f"  Example: {table}.{column}")

# Method 1: trace_column_forward() - Final impacts only
print("\n  Method 1: trace_column_forward()")
print("  Returns only the final impacted columns (leaf nodes with no downstream).\n")
try:
    impacts = pipeline.trace_column_forward(table, column)
    if impacts:
        print("  Code: impacts = pipeline.trace_column_forward(table, column)")
        print("  Result:")
        for impact in impacts[:8]:
            print(f"    → {impact.table_name}.{impact.column_name}")
        if len(impacts) > 8:
            print(f"    ... and {len(impacts) - 8} more")
    else:
        print("    (no downstream impacts)")
except Exception as e:
    print(f"    Error: {e}")

# Method 2: get_table_impact_path() - Table-level path
print("\n  Method 2: get_table_impact_path()")
print("  Returns the complete impact path through real tables (skips CTEs).\n")
try:
    path = pipeline.get_table_impact_path(table, column)
    if path:
        print("  Code: path = pipeline.get_table_impact_path(table, column)")
        print("  Result:")
        for i, (tbl, col, _query_id) in enumerate(path):
            if i == 0:
                prefix = "    "
                arrow = ""
            else:
                prefix = "    ├── " if i < len(path) - 1 else "    └── "
                arrow = "→ "
            print(f"{prefix}{arrow}{tbl}.{col}")
        print(f"\n  Total: {len(path)} columns in impact path")
    else:
        print("    (no path found)")
except Exception as e:
    print(f"    Error: {e}")

# Method 3: trace_column_forward_full() - Full transparency with CTEs
print("\n  Method 3: trace_column_forward_full()")
print("  Returns all nodes and edges, including CTEs for complete transparency.\n")
try:
    nodes, edges = pipeline.trace_column_forward_full(table, column, include_ctes=True)
    if nodes:
        print("  Code: nodes, edges = pipeline.trace_column_forward_full(table, column)")
        print("  Result (nodes - first 10):")
        for node in nodes[:10]:
            layer_tag = " [CTE]" if node.layer == "cte" else ""
            cross_query = " [cross-query]" if node.layer == "input" else ""
            print(f"    • {node.table_name}.{node.column_name}{layer_tag}{cross_query}")
        if len(nodes) > 10:
            print(f"    ... and {len(nodes) - 10} more nodes")

        print("\n  Result (edges - first 8):")
        for edge in edges[:8]:
            edge_desc = f"({edge.edge_type})"
            print(
                f"    {edge.from_node.table_name}.{edge.from_node.column_name} "
                f"→ {edge.to_node.table_name}.{edge.to_node.column_name} {edge_desc}"
            )
        if len(edges) > 8:
            print(f"    ... and {len(edges) - 8} more edges")
    else:
        print("    (no path found)")
except Exception as e:
    print(f"    Error: {e}")

# Show additional examples with just the simple method
print("  Additional examples (using trace_column_forward for brevity):")
print()

for table, column in impact_examples[1:]:
    print(f"  Column: {table}.{column}")
    try:
        impacts = pipeline.trace_column_forward(table, column)
        if impacts:
            print("  Final impacted columns:")
            for impact in impacts[:5]:
                print(f"    → {impact.table_name}.{impact.column_name}")
            if len(impacts) > 5:
                print(f"    ... and {len(impacts) - 5} more")
        else:
            print("    (no downstream impacts)")
    except Exception as e:
        print(f"    Error: {e}")
    print()

5. FORWARD LINEAGE / IMPACT ANALYSIS (Three levels of detail)

graph provides three methods for forward lineage (impact analysis):

Method                       | Returns                      | Use Case                            |
-----------------------------|------------------------------|-------------------------------------|
trace_column_forward()       | Final impacted columns only  | Quick: "what does this affect?"     |
get_table_impact_path()      | Full table path (no CTEs)    | "Show complete impact through tables"|
trace_column_forward_full()  | All nodes + edges (with CTEs)| Full transparency for debugging     |

  Example: raw_orders.total_amount

  Method 1: trace_column_forward()
  Returns only the final impacted columns (leaf nodes with no downstream).

  Code: impacts = pipeline.trace_column_forward(table, column)
  Result:
    → stg_orders_enriched.customer_prior_spend
    → int_daily_metrics.gross_revenue
    → int_daily_metrics.revenue_dod_growth_pct
    → int_dail

### Section

In [8]:
print("6. PII COLUMN TRACKING")

# Mark known PII columns (use actual table names)
pii_columns = [
    ("raw_customers", "email"),
    ("raw_customers", "phone_number"),
    ("raw_orders", "ip_address"),
    ("raw_orders", "shipping_address"),
]

marked_count = 0
for table, column in pii_columns:
    # Find columns by table_name and column_name (keys include query_id prefix)
    for col in pipeline.columns.values():
        if col.table_name == table and col.column_name == column:
            col.pii = True
            marked_count += 1
print(f"  Marked {marked_count} columns as PII")

# Propagate PII metadata through lineage
pipeline.propagate_all_metadata()

# Find all PII columns
all_pii = list(pipeline.get_pii_columns())
print(f"  PII columns in pipeline ({len(all_pii)}):")
for col in sorted(all_pii, key=lambda c: f"{c.table_name}.{c.column_name}"):
    print(f"    ⚠️  {col.table_name}.{col.column_name}")
print()

6. PII COLUMN TRACKING
  Marked 4 columns as PII
📊 Pass 1: Propagating metadata backward from 172 output columns...
📊 Pass 2: Propagating metadata forward for 172 columns...
✅ Done! Propagated metadata for 172 columns
  PII columns in pipeline (16):
    ⚠️  raw_customers.email
    ⚠️  raw_customers.first_name
    ⚠️  raw_customers.last_name
    ⚠️  raw_customers.phone_number
    ⚠️  raw_orders.ip_address
    ⚠️  raw_orders.shipping_address
    ⚠️  source_customers.email
    ⚠️  source_customers.first_name
    ⚠️  source_customers.last_name
    ⚠️  source_customers.phone_number
    ⚠️  source_orders.ip_address
    ⚠️  source_orders.shipping_address
    ⚠️  stg_orders_enriched.customer_email
    ⚠️  stg_orders_enriched.customer_first_name
    ⚠️  stg_orders_enriched.customer_full_name
    ⚠️  stg_orders_enriched.customer_last_name



### Section

In [9]:
print("7. TABLE GRAPH METHODS (pipeline.table_graph)")
print("  Demonstrates available methods on the TableDependencyGraph.\n")

# 7a. Get source tables (external tables not created by any query)
print("  7a. Source Tables (external inputs):")
source_tables = pipeline.table_graph.get_source_tables()
for table in source_tables[:10]:
    print(f"      📥 {table.table_name}")
if len(source_tables) > 10:
    print(f"      ... and {len(source_tables) - 10} more")
print()

# 7b. Get final tables (not read by any downstream query)
print("  7b. Final Tables (pipeline outputs):")
final_tables = pipeline.table_graph.get_final_tables()
for table in final_tables[:10]:
    print(f"      📤 {table.table_name}")
if len(final_tables) > 10:
    print(f"      ... and {len(final_tables) - 10} more")
print()

# 7c. Get execution order (tables in dependency order)
print("  7c. Table Execution Order:")
execution_order = pipeline.table_graph.get_execution_order()
for i, table in enumerate(execution_order[:15], 1):
    table_type = "source" if table.is_source else "created"
    print(f"      {i}. {table.table_name} ({table_type})")
if len(execution_order) > 15:
    print(f"      ... and {len(execution_order) - 15} more")
print()

# 7d. Get dependencies for a specific table
print("  7d. Table Dependencies (upstream tables):")
example_tables = ["mart_customer_ltv", "int_daily_metrics"]
for table_name in example_tables:
    if table_name in pipeline.table_graph.tables:
        deps = pipeline.table_graph.get_dependencies(table_name)
        print(f"      {table_name} depends on:")
        if deps:
            for dep in deps[:5]:
                print(f"        ← {dep.table_name}")
            if len(deps) > 5:
                print(f"        ... and {len(deps) - 5} more")
        else:
            print("        (no dependencies)")
print()

# 7e. Get downstream tables
print("  7e. Table Downstream (impact analysis):")
example_sources = ["raw_orders", "raw_customers"]
for table_name in example_sources:
    if table_name in pipeline.table_graph.tables:
        downstream = pipeline.table_graph.get_downstream(table_name)
        print(f"      {table_name} is used by:")
        if downstream:
            for ds in downstream[:5]:
                print(f"        → {ds.table_name}")
            if len(downstream) > 5:
                print(f"        ... and {len(downstream) - 5} more")
        else:
            print("        (no downstream tables)")
print()

# 7f. Access table and query objects directly
print("  7f. Direct Access to Tables and Queries:")
print(f"      Total tables: {len(pipeline.table_graph.tables)}")
print(f"      Total queries: {len(pipeline.table_graph.queries)}")
print()
print("      Example table access:")
if "raw_orders" in pipeline.table_graph.tables:
    table = pipeline.table_graph.tables["raw_orders"]
    print("        table = pipeline.table_graph.tables['raw_orders']")
    print(f"        table.is_source = {table.is_source}")
    print(f"        table.read_by = {table.read_by[:3]}{'...' if len(table.read_by) > 3 else ''}")
print()
print("      Example query access:")
first_query_id = list(pipeline.table_graph.queries.keys())[0]
first_query = pipeline.table_graph.queries[first_query_id]
print(f"        query = pipeline.table_graph.queries['{first_query_id}']")
print(f"        query.destination_table = {first_query.destination_table}")
print(
    f"        query.source_tables = {list(first_query.source_tables)[:3]}{'...' if len(first_query.source_tables) > 3 else ''}"
)
print()

7. TABLE GRAPH METHODS (pipeline.table_graph)
  Demonstrates available methods on the TableDependencyGraph.

  7a. Source Tables (external inputs):
      📥 source_orders
      📥 source_customers
      📥 source_products
      📥 source_order_items
      📥 order_items_agg
      📥 customer_orders_prior
      📥 daily_with_running
      📥 daily_orders
      📥 customer_rfm
      📥 customer_metrics
      ... and 4 more

  7b. Final Tables (pipeline outputs):
      📤 int_daily_metrics
      📤 mart_customer_ltv
      📤 mart_product_performance

  7c. Table Execution Order:
      1. source_orders (source)
      2. source_customers (source)
      3. source_products (source)
      4. source_order_items (source)
      5. order_items_agg (source)
      6. customer_orders_prior (source)
      7. daily_with_running (source)
      8. daily_orders (source)
      9. customer_rfm (source)
      10. customer_metrics (source)
      11. category_totals (source)
      12. product_aggregates (source)
      13. 

### Section

In [10]:
print("8. COLUMN GRAPH METHODS (pipeline.column_graph)")
print("  Demonstrates available methods on the PipelineLineageGraph.\n")

# 8a. Get source columns (no incoming edges)
print("  8a. Source Columns (external inputs):")
source_columns = pipeline.column_graph.get_source_columns()
for col in source_columns[:10]:
    print(f"      📥 {col.full_name}")
if len(source_columns) > 10:
    print(f"      ... and {len(source_columns) - 10} more")
print()

# 8b. Get final columns (no outgoing edges)
print("  8b. Final Columns (pipeline outputs):")
final_columns = pipeline.column_graph.get_final_columns()
for col in final_columns[:10]:
    print(f"      📤 {col.full_name}")
if len(final_columns) > 10:
    print(f"      ... and {len(final_columns) - 10} more")
print()

# 8c. Get upstream columns (direct dependencies)
print("  8c. Column Upstream (direct dependencies):")
example_cols = ["mart_customer_ltv.lifetime_revenue", "int_daily_metrics.gross_revenue"]
for col_name in example_cols:
    if col_name in pipeline.columns:
        upstream = pipeline.column_graph.get_upstream(col_name)
        print(f"      {col_name} depends on:")
        if upstream:
            for up in upstream[:5]:
                print(f"        ← {up.full_name}")
            if len(upstream) > 5:
                print(f"        ... and {len(upstream) - 5} more")
        else:
            print("        (no direct dependencies)")
print()

# 8d. Get downstream columns (direct dependents)
print("  8d. Column Downstream (direct impact):")
example_sources = ["raw_orders.total_amount", "raw_customers.customer_id"]
for col_name in example_sources:
    if col_name in pipeline.columns:
        downstream = pipeline.column_graph.get_downstream(col_name)
        print(f"      {col_name} is used by:")
        if downstream:
            for ds in downstream[:5]:
                print(f"        → {ds.full_name}")
            if len(downstream) > 5:
                print(f"        ... and {len(downstream) - 5} more")
        else:
            print("        (no direct dependents)")
print()

# 8e. Build column dependencies (graphlib format)
print("  8e. Column Dependencies (graphlib format):")
col_deps = pipeline.column_graph._build_column_dependencies()
print(f"      Total columns: {len(col_deps)}")
# Show a sample
sample_col = next((c for c in col_deps if len(col_deps[c]) > 0), None)
if sample_col:
    print(f"      Example: {sample_col}")
    print(
        f"        depends on: {list(col_deps[sample_col])[:3]}{'...' if len(col_deps[sample_col]) > 3 else ''}"
    )
print()

# 8f. Direct access to columns and edges
print("  8f. Direct Access to Columns and Edges:")
print(f"      Total columns: {len(pipeline.column_graph.columns)}")
print(f"      Total edges: {len(pipeline.column_graph.edges)}")
print()
print("      Example column access:")
first_col_name = next(iter(pipeline.column_graph.columns.keys()), None)
if first_col_name:
    first_col = pipeline.column_graph.columns[first_col_name]
    print(f"        col = pipeline.column_graph.columns['{first_col_name}']")
    print(f"        col.column_name = {first_col.column_name}")
    print(f"        col.table_name = {first_col.table_name}")
print()
print("      Example edge access:")
if pipeline.column_graph.edges:
    first_edge = pipeline.column_graph.edges[0]
    print("        edge = pipeline.column_graph.edges[0]")
    print(f"        edge.from_node = {first_edge.from_node.full_name}")
    print(f"        edge.to_node = {first_edge.to_node.full_name}")
    print(f"        edge.edge_type = {first_edge.edge_type}")
print()

8. COLUMN GRAPH METHODS (pipeline.column_graph)
  Demonstrates available methods on the PipelineLineageGraph.

  8a. Source Columns (external inputs):
      📥 source_orders.order_id
      📥 source_orders.customer_id
      📥 source_orders.order_date
      📥 source_orders.order_timestamp
      📥 source_orders.status
      📥 source_orders.shipping_address
      📥 source_orders.payment_method
      📥 source_orders.subtotal_amount
      📥 source_orders.tax_amount
      📥 source_orders.shipping_amount
      ... and 41 more

  8b. Final Columns (pipeline outputs):
      📤 raw_customers.customer_id
      📤 raw_customers.phone_number
      📤 raw_customers.created_at
      📤 raw_products.product_id
      📤 raw_products.is_active
      📤 raw_products.created_at
      📤 raw_order_items.order_item_id
      📤 raw_order_items.created_at
      📤 05_stg_orders_enriched:cte:order_items_agg.order_id
      📤 05_stg_orders_enriched:cte:customer_orders_prior.order_id
      ... and 121 more

  8c. Column Ups

### Section

In [11]:
print("PIPELINE SUMMARY")
print(f"  Total queries:  {len(queries)}")
print(f"  Total tables:   {len(pipeline.table_graph.tables)}")
print(f"  Total columns:  {len(pipeline.columns)}")
print(f"  Total edges:    {len(pipeline.edges)}")
print(f"  PII columns:    {len(list(all_pii))}")
print()
print("  Graph Objects:")
print("    pipeline.table_graph  -> TableDependencyGraph (tables + queries)")
print("    pipeline.column_graph -> PipelineLineageGraph (columns + edges)")
print()
print("Analysis complete!")

PIPELINE SUMMARY
  Total queries:  8
  Total tables:   22
  Total columns:  326
  Total edges:    350
  PII columns:    16

  Graph Objects:
    pipeline.table_graph  -> TableDependencyGraph (tables + queries)
    pipeline.column_graph -> PipelineLineageGraph (columns + edges)

Analysis complete!
